# CIS 700 Final Project Colab file
Authors: Dhruv Desai, Rahul Shekhar

In [0]:
!git clone https://github.com/dhruvd25/cis700_project.git
!mv cis700_project/helper.py .
!mv cis700_project/celebloader.py .
!mv cis700_project/Generator6 .

In [0]:
import numpy as np
from datetime import datetime, timedelta
from google.colab import drive
from scipy.io import loadmat
from scipy.io import savemat
import torchvision.transforms as transforms
import torch
from importlib.machinery import SourceFileLoader
from torch.autograd import Variable
from torch.nn import functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision
from helper import Logger
from celebloader import celebrity
# CUDA for PyTorch
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [0]:
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

!if [ -f ngrok ] ; then echo "Ngrok already installed" ; else wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1 && unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1 ; fi

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
!rm -r ./logs

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard Link: ' +str(json.load(sys.stdin)['tunnels'][0]['public_url']))"

In [0]:
mat_file_loc = '/content/cis700_project/wiki_edit.mat'
root_dir_loc = '/content/cis700_project/wiki_crop'

## Sampled images from Dataset

In [0]:
input_size = 64
input_transforms = transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor()])

dataset = celebrity(landmarks_frame = mat_file_loc
                                ,root_dir=root_dir_loc,
                                transform=input_transforms)
train_loader = torch.utils.data.DataLoader(dataset, batch_size = 8,
                                           shuffle = True)
train_iter = iter(train_loader)
images, labels = train_iter.next()
grid = torchvision.utils.make_grid(images)
plt.figure(figsize=(10,10))
plt.imshow(grid.numpy().transpose((1, 2, 0)))
plt.axis('off')
plt.title(labels.numpy())

## Non-Deep Learning benchmark for classification (Logistic Regression model)

In [0]:
input_size = 64
num_classes = 2
num_epochs = 5
learning_rate = 0.001
  
class LogisticRegression(torch.nn.Module):
  def __init__(self):
    super(LogisticRegression,self).__init__()
    self.linear = torch.nn.Linear(64*64*3,3)
  def forward(self,x):
    x = x.view(x.size(0), -1)
    out = self.linear(x)
    return out

model = LogisticRegression().to(device)
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

logger = Logger('./logs/'+str(1))

steps = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        if i < 286:
          # Load images as Variable
          images = images.view(-1, 64*64*3).requires_grad_().to(device)
          labels = labels.to(device)
          # Clear gradients w.r.t. parameters
          optimizer.zero_grad()

          # Forward pass to get output/logits
          outputs = model(images)

          # Calculate Loss: softmax --> cross entropy loss
          loss = criterion(outputs, labels)

          # Getting gradients w.r.t. parameters
          loss.backward()

          # Updating parameters
          optimizer.step()

          _, argmax = torch.max(outputs, 1)
          accuracy = (labels == argmax.squeeze()).float().mean()
          if steps % 200 == 0:
            print("Loss:",loss.item(),'accuracy', accuracy.item())
          # 1. Log scalar values (scalar summary)
          info = { 'loss': loss.item(), 'accuracy': accuracy.item() }
          for tag, value in info.items():
              logger.scalar_summary(tag, value, steps+1)

          steps += 1

# Calculate Accuracy         
correct = 0
total = 0
# Iterate through test dataset
for i,(images, labels) in enumerate(train_loader):
    if i>286:
      # Load images to a Torch Variable
      images = images.view(-1, 64*64*3).requires_grad_().to(device)

      # Forward pass only to get logits/output
      outputs = model(images).to(device)

      # Get predictions from the maximum value
      _, predicted = torch.max(outputs.data, 1)

      # Total number of labels
      total += labels.size(0)

      # Total correct predictions
      correct += (predicted.cpu() == labels.cpu()).sum()

accuracy = 100 * correct / total

# Print Loss
print(' Loss: {}. Accuracy: {}'.format( loss.item(), accuracy))

## Deep Learning Benchmark for classification (CNN)

In [0]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    
    self.layer1 = nn.Sequential(
                  nn.Conv2d(3, 16, 3, padding = 1),
                  nn.BatchNorm2d(16),
                  nn.ReLU(),
                  nn.MaxPool2d(2,2))
    self.layer2 = nn.Sequential(
                  nn.Conv2d(16, 32, 3, padding = 1),
                  nn.BatchNorm2d(32),
                  nn.ReLU(),
                  nn.MaxPool2d(2,2))
    self.layer3 = nn.Sequential(
                  nn.Conv2d(32, 64, 3, padding = 1),
                  nn.BatchNorm2d(64),
                  nn.ReLU(),
                  nn.MaxPool2d(2,2))
    self.layer4 = nn.Sequential(
                  nn.Conv2d(64, 128, 3, padding = 1),
                  nn.BatchNorm2d(128),
                  nn.ReLU(),
                  nn.MaxPool2d(2,2))    
    
    self.fc1 = nn.Linear(2048,256)
    self.fc2 = nn.Linear(256,2)
    self.softmax = nn.LogSoftmax(dim=1)

    
    self.dropout = nn.Dropout(0.3)
    
  def forward(self,x):
    
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = x.view(x.size(0), -1)
    x = self.dropout(x)
    x = F.relu(self.fc1(x))
    x = self.dropout(x)
    x = self.fc2(x)
    
    return self.softmax(x)

In [0]:
num_epochs = 4
learning_rate = 0.0001

model = CNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
logger = Logger('./logs/'+ str(2))
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
steps = 0

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    print(epoch)
    for i, (images, labels) in enumerate(train_loader):
        if i <= 286:
          # Move tensors to the configured device
          images = images.to(device)
          labels = labels.to(device)

          # Forward pass
          outputs = model(images)
          loss = criterion(outputs, labels)

          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          _, argmax = torch.max(outputs, 1)
          accuracy = (labels == argmax.squeeze()).float().mean()
          if (i+1) % 200 == 0:
              print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
          # 1. Log scalar values (scalar summary)
          info = { 'loss': loss.item(), 'accuracy': accuracy.item() }
          for tag, value in info.items():
              logger.scalar_summary(tag, value, steps+1)
          steps += 1
print("Training accuracy:({:.0f}%)".format(100. * accuracy.item()))

test_loss = 0
correct = 0
with torch.no_grad():
    for i,(images, labels) in enumerate(train_loader):
        if i >286:      
          images, labels = images.to(device), labels.to(device)
          output = model(images)
          # sum up batch loss
          test_loss += F.nll_loss(output, labels, reduction='sum').item() 
          # get the index of the max log-probability
          pred = output.argmax(dim=1, keepdim=True) 
          correct += pred.eq(labels.view_as(pred)).sum().item()

          _, argmax = torch.max(output, 1)
          accuracy = (labels == argmax.squeeze()).float().mean()
test_loss /= 146 
print('\nTest set:Accuracy: {}/{} ({:.0f}%)\n'
      .format(correct,(146*32),100. * correct / (146*32) ))

## Advanced Deep Learning Architecture 
Conditional DCGAN (cGAN)

In [0]:
class generator(nn.Module):
  def __init__(self):
    super(generator, self).__init__()
    f = 512
    self.main = nn.Sequential(
      nn.ConvTranspose2d(258, f, 4, 
                         stride = 1, padding = 0 ,bias=False),  #4, 512
      nn.BatchNorm2d(f),
      nn.ReLU(True),
      nn.ConvTranspose2d(f, f//4, 4, 
                         stride = 2, padding = 1,bias=False), # 8, 128
      nn.BatchNorm2d(f//4), 
      nn.ReLU(True),
      nn.ConvTranspose2d(f//4, f//16, 4, 
                         stride = 2, padding = 1,bias=False), #16, 32
      nn.BatchNorm2d(f//16),
      nn.ReLU(True),
      nn.ConvTranspose2d(f//16, f//64, 4, 
                         stride = 2, padding = 1,bias=False), #32, 8
      nn.BatchNorm2d(f//64), 
      nn.ReLU(True),
      nn.ConvTranspose2d(f//64, 3, 4, 
                         stride = 2, padding = 1,bias=False), #64,  1
      nn.Tanh())  
  def forward(self, x):
    out = self.main(x)
    return out

class discriminator(nn.Module):
  def __init__(self):
    super(discriminator, self).__init__()
    d = 8
    self.conv1 = nn.Conv2d(3, d, 4, stride = 2, padding = 1,bias=False)
    
    self.main = nn.Sequential(
      nn.Conv2d(d + 2, d*2, 4, stride = 2, padding = 1,bias=False), # 16
      nn.BatchNorm2d(d*2),
      nn.LeakyReLU(0.2),
      nn.Conv2d(d*2, d*4, 4, stride = 2, padding = 1,bias=False), # 8
      nn.BatchNorm2d(d*4),
      nn.LeakyReLU(0.2),
      nn.Conv2d(d*4, d*8, 4, stride = 2, padding = 1,bias=False), # 4
      nn.LeakyReLU(0.2),
      nn.BatchNorm2d(d*8),
      nn.Conv2d(d*8, 1, 4, stride = 1, padding = 0,bias=False), # 1
      nn.Sigmoid())
  def forward(self, x, y):
    x_1 = self.conv1(x)
    x_c = torch.cat((x_1, y), dim = 1)
    out = self.main(x_c)
    return out
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(0.0, 0.02)
        m.bias.data.fill_(0)

### GAN parameters

In [0]:
input_size = 64
input_transforms = transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])
dataset = celebrity(landmarks_frame = mat_file_loc
                                ,root_dir=root_dir_loc,
                                transform=input_transforms)
train_loader = torch.utils.data.DataLoader(dataset, batch_size = 64,
                                           shuffle = True)

In [0]:
NUM_CLASSES = 2

G = generator().to(device)
D = discriminator().to(device)

G.apply(weights_init)
D.apply(weights_init)

num_epochs, D_step, G_step, step = 25, 0, 0, 0
logger = Logger('./logs/'+ str(3))

In [0]:
#Working model params
optimizer_G = torch.optim.Adam(G.parameters(), lr=0.001, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(D.parameters(), lr=0.0005, betas = (0.5, 0.999))
criterion = nn.BCELoss()
real_label, fake_label = 0.90, 0.0

### Training Loop for GAN

In [0]:
!mkdir GAN_Training

In [0]:
for epoch in range(num_epochs):
  print(epoch)
  for images, labels in train_loader:
    
    images = images.to(device)
    sz = len(images)
        
    fake_D = (fake_label + torch.rand(sz) * 0.1).to(device)  
    real_D = (real_label + torch.rand(sz) * 0.1).to(device) 
    
    fake_G = torch.zeros(sz).to(device)  
    real_G = torch.ones(sz).to(device)
    
    
    #images += (torch.randn(images.size()) * 0.1).to(device)
    #(torch.randn(G_noise_discriminator.size()) * 0.1).to(device)
    
    
    # Discriminator (REAL)
    #Create Discriminator concatenated 
    d_condition = torch.zeros(sz, NUM_CLASSES, 32, 32).to(device)
    idx = torch.arange(sz).to(device)
    d_condition[idx, labels] = 1
    
    
    D.zero_grad()
    D_real_output = D(images, d_condition).squeeze()
    
    
    # TO MODIFY (USING ACTUAL LABELS)
    z = torch.randn(sz, 256, 1, 1, device=device)
    y_c = (torch.rand(sz, 1) * NUM_CLASSES).type(torch.LongTensor).to(device)
    y_label = torch.zeros(sz, NUM_CLASSES).to(device)
    y_label.scatter_(1, y_c.view(sz, 1), 1)
    y_label = y_label.reshape(sz, NUM_CLASSES, 1, 1)
    final_z_vec_D = torch.cat((z, y_label), dim = 1).to(device)
    
    d_condition = torch.zeros(sz, NUM_CLASSES, 32, 32).to(device)
    idx = torch.arange(sz).to(device)
    d_condition[idx, y_c.squeeze()] = 1
      
    G_noise_discriminator = G(final_z_vec_D).to(device)
    
    D_fake_output = D(G_noise_discriminator, d_condition).squeeze()
    
    D_real_loss = criterion(D_real_output, real_D)
    D_fake_loss = criterion(D_fake_output, fake_D)

    
    D_loss = (D_real_loss + D_fake_loss)/2
    D_loss.backward()
    optimizer_D.step()
    

    # GENERATOR
    G.zero_grad() #equal to detaching the Generator
    z = torch.randn(sz, 256, 1, 1, device=device).to(device)
    
    
    #adding condition to noise vector
    y_c = (torch.rand(sz, 1) * NUM_CLASSES).type(torch.LongTensor).to(device)
    y_label = torch.zeros(sz, NUM_CLASSES).to(device)
    y_label.scatter_(1, y_c.view(sz, 1), 1)
    y_label = y_label.reshape(sz, NUM_CLASSES, 1, 1)
      
    final_z_vec_G = torch.cat((z, y_label), dim = 1).to(device)

    d_condition = torch.zeros(sz, NUM_CLASSES, 32, 32).to(device)
    idx = torch.arange(sz).to(device)
    d_condition[idx, y_c.squeeze()] = 1
    
    
    G_noise_generator = G(final_z_vec_G).to(device)
    D_generator_output = D(G_noise_generator, d_condition).squeeze()
    G_loss = criterion(D_generator_output, real_G)
    G_loss.backward()
    optimizer_G.step()
    
    
    info = {'Total D Loss': D_loss.item(), 'D Real Loss': D_real_loss.item(), 
            'D Fake Loss': D_fake_loss.item(), 'G Loss': G_loss.item()}
    for tag, value in info.items():
      logger.scalar_summary(tag, value, step + 1)
    
    if step % 500 == 0:
      print('saved')
      torchvision.utils.save_image(G_noise_generator.data, 'GAN_Training/' + 
                                   str(step)+ '.png', normalize=True)
      torchvision.utils.save_image(images.data, 'GAN_Training/Original' + 
                             str(step)+ '.png', normalize=True)
      
    step += 1

In [0]:
!rm -r GAN_Training

In [0]:
# Saving and downloading model weights
from google.colab import files
torch.save(G.state_dict(), "Generator")
torch.save(D.state_dict(), "Disc")
files.download('Generator')
files.download('Disc')

In [0]:
# Load trained Generator model
model = generator().to(device)
model.load_state_dict(torch.load('Generator6'))

## Creating synthetic dataset using trained Generator

In [0]:
synthetic_data = []
NUM_CLASSES = 2
#Creating synthetic dataset
for i in range(100000):
  sz = 1
  z = torch.randn(sz, 256, 1, 1, device=device).to(device)
  #adding condition to noise vector
  y_c = (torch.rand(sz, 1) * NUM_CLASSES).type(torch.LongTensor).to(device)
  y_label = torch.zeros(sz, NUM_CLASSES).to(device)
  y_label.scatter_(1, y_c.view(sz, 1), 1)
  y_label = y_label.reshape(sz, NUM_CLASSES, 1, 1)
  final_z_vec_G = torch.cat((z, y_label), dim = 1).to(device)
  
  G_noise_generator = model(final_z_vec_G).to(device)
  
  synthetic_data.append([G_noise_generator.data, z])

In [0]:
train_loader_E = torch.utils.data.DataLoader(synthetic_data, batch_size = 64,
                                           shuffle = True)

## Encoder Architecture

In [0]:
class encoder(nn.Module):
  def __init__(self):
    super(encoder, self).__init__()
    self.encode = nn.Sequential(
      nn.Conv2d(3, 32, 5, stride = 2), 
      nn.BatchNorm2d(32),
      nn.ReLU(),
      #nn.MaxPool2d(2, stride = 2), 
      nn.Conv2d(32, 64, 5, stride = 2), 
      nn.BatchNorm2d(64),
      nn.ReLU(),
      #nn.MaxPool2d(2, stride = 2),
      nn.Conv2d(64, 128, 5, stride = 2),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      #nn.MaxPool2d(2, stride = 2), 
      nn.Conv2d(128, 256, 5, stride = 2),
      nn.BatchNorm2d(256),
      nn.ReLU(),
      #nn.MaxPool2d(2, stride = 2) # 256 * 1 * 1 
    )
    
    self.fc1 = nn.Linear(256, 4096)
    self.bc1 = nn.BatchNorm1d(4096)
    self.fc2 = nn.Linear(4096, 256)
    
  def forward(self, x):
    out = self.encode(x)
    out = out.view(out.size(0), -1)
    out = F.relu(self.bc1(self.fc1(out)))
    out = self.fc2(out)
    
    return out.view(out.size(0), 256, 1, 1)

### Encoder training 

In [0]:
!mkdir Encoder_Training

In [0]:
E = encoder().to(device)
# E.apply(weights_init)
criterion_E = nn.MSELoss()
optimizer_E = torch.optim.Adam(E.parameters(), 
                               lr = 0.0002, betas = (0.5, 0.999))

num_epochs, step = 25, 0

logger = Logger('./logs/'+ str(4))

for epoch in range(num_epochs):
  print('epoch: ' + str(epoch))
  for i, (images, labels) in enumerate(train_loader_E):
    images = images.squeeze().to(device)
    output = E(images)
    labels = labels.reshape(output.shape)
          
    loss = criterion_E(output, labels)
    
    optimizer_E.zero_grad()
    loss.backward()
    optimizer_E.step()

    if step % 200 == 0:
      print(loss.item())
    info = {'loss': loss.item()}
    for tag, value in info.items():
      logger.scalar_summary(tag, value, step + 1)
    torchvision.utils.save_image(images.data, 'Encoder_Training/' + 
                                 str(i)+ '.png', normalize=True)
    
    step += 1


In [0]:
!rm -r Encoder_Training

### Comparing outputs from trained GAN with and without Encoder

In [0]:
!mkdir Latent_Training

In [0]:
G_final = generator().to(device)
G_final.load_state_dict(torch.load("Generator6"))
NUM_CLASSES = 2
for i, (images, labels) in enumerate(train_loader):
  sz = len(images)
  images = images.to(device)
  G_input = E(images)
  y_c = (torch.rand(sz, 1) * NUM_CLASSES).type(torch.LongTensor).to(device)
  y_label = torch.zeros(sz, NUM_CLASSES).to(device)
  y_label.scatter_(1, y_c.view(sz, 1), 1)
  y_label = y_label.reshape(sz, NUM_CLASSES, 1, 1)
  
  final_E_vec_G = torch.cat((G_input, y_label), dim = 1).to(device)
  
   
  #z = torch.randn(sz, 256, 1, 1, device=device).to(device)
  #final_z_vec_G = torch.cat((z, y_label), dim = 1).to(device)

  G_E_generator = G_final(final_E_vec_G).to(device)
  G_Z_generator = G_final(final_z_vec_G).to(device)
  torchvision.utils.save_image(G_E_generator.data, 'Latent_Training/E' 
                               + str(i)+ '.png', normalize=True)
  torchvision.utils.save_image(G_Z_generator.data, 'Latent_Training/N' 
                               + str(i)+ '.png', normalize=True)
  torchvision.utils.save_image(images.data, 'Latent_Training/Original' 
                               + str(i)+ '.png', normalize=True)
  
  if i > 10:
    break

In [0]:
!rm -r Latent_Training

## Code to see output for different labels but same noise vector


In [0]:
!mkdir Age

In [0]:
G_final = generator().to(device)
G_final.load_state_dict(torch.load("Generator6"))
NUM_CLASSES = 2

for i in range(5):
  sz = 64
  z = torch.randn(sz, 256, 1, 1, device=device).to(device)
  
  #y_c = (torch.rand(sz, 1) * NUM_CLASSES).type(torch.LongTensor).to(device)
  
  y_c0 = torch.tensor([[0] * 64]).type(torch.LongTensor).to(device)
  y_c1 = torch.tensor([[1] * 64]).type(torch.LongTensor).to(device)
  
  y_label0 = torch.zeros(sz, NUM_CLASSES).to(device)
  y_label0.scatter_(1, y_c0.view(sz, 1), 1)
  y_label0 = y_label0.reshape(sz, NUM_CLASSES, 1, 1)
  y_0_cat = torch.cat((z, y_label0), dim = 1).to(device)
  

  y_label1 = torch.zeros(sz, NUM_CLASSES).to(device)
  y_label1.scatter_(1, y_c1.view(sz, 1), 1)
  y_label1 = y_label1.reshape(sz, NUM_CLASSES, 1, 1)
  y_1_cat = torch.cat((z, y_label1), dim = 1).to(device)
  
  
  G_E_generator0 = G_final(y_0_cat).to(device)
  G_E_generator1 = G_final(y_1_cat).to(device)
  
  torchvision.utils.save_image(G_E_generator0.data, 'Age/' 
                               + str(i)+'Young'+ '.png', normalize=True)
  torchvision.utils.save_image(G_E_generator1.data, 'Age/' 
                               + str(i)+ 'Old'+'.png', normalize=True)
  
  

In [0]:
!rm -r Age